In [2]:
import pandas as pd
import nltk
from rouge_score import rouge_scorer
import warnings
from sklearn.metrics.pairwise import cosine_similarity


warnings.filterwarnings("ignore")

# Function to calculate BLEU score using NLTK
def calculate_bleu(references, candidates):
    bleu_scores = []
    for ref, cand in zip(references, candidates):
        bleu_score = nltk.translate.bleu_score.sentence_bleu([nltk.word_tokenize(ref)], nltk.word_tokenize(cand))
        bleu_scores.append(bleu_score)
    return sum(bleu_scores) / len(bleu_scores)  # Return average BLEU score

In [2]:
def nearest_house_distance(N, house_positions, house_widths, M, people_positions):
    # Create a list of ranges for each house
    house_ranges = [
        (pos, pos + width - 1)
        for pos, width in zip(house_positions, house_widths)
    ]
    
    total_distance = 0
    
    # For each person, calculate the minimum distance to the nearest house
    for person in people_positions:
        min_distance = float('inf')
        
        for house_start, house_end in house_ranges:
            if house_start <= person <= house_end:
                # Person is already inside the house
                min_distance = 0
                break
            else:
                # Calculate distance to the nearest point of the house
                distance = min(abs(person - house_start), abs(person - house_end))
                min_distance = min(min_distance, distance)
        
        total_distance += min_distance
    
    return total_distance

# Example input
N = 2
house_positions = [15,43]
house_widths = [15,2]
M = 3
people_positions = [30,4,44]

# Output the result
result = nearest_house_distance(N, house_positions, house_widths, M, people_positions)
print(result)  # Output: 5

12


In [1]:
!pwd


/Users/co/major_project


In [2]:
# import numpy as np
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize


# def cosine_similarity(text1, text2):
#     """
#     Calculates the cosine similarity between two arrays of strings.

#     Args:
#         text1 (list): A list of strings.
#         text2 (list): A list of strings.

#     Returns:
#         float: The cosine similarity between the two input arrays.
#     """

#     # Preprocessing
#     stop_words = stopwords.words('english')

#     def preprocess(text):
#         tokens = word_tokenize(text.lower())
#         return [w for w in tokens if w not in stop_words]

#     vectorizer = np.vectorize(preprocess)  # Efficiently preprocess lists of strings
#     text1_processed = vectorizer(text1)
#     text2_processed = vectorizer(text2)

#     # Create common vocabulary
#     all_words = np.unique(np.concatenate([text1_processed, text2_processed]))

#     # Create vectors
#     def text_to_vector(text):
#         vector = np.zeros(len(all_words))
#         for w in text:
#             if w in all_words:
#                 vector[np.where(all_words == w)[0][0]] = 1
#         return vector

#     vector1 = text_to_vector(text1_processed)
#     vector2 = text_to_vector(text2_processed)

#     # Calculate cosine similarity
#     cosine = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

#     return cosine


In [7]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def cosine_similarity(text1, text2):
    """
    Calculates the cosine similarity between two arrays of strings. Handles cases
    where the input consists of lists of lists of words.

    Args:
        text1 (list): A list of strings or a list of lists of words.
        text2 (list): A list of strings or a list of lists of words.

    Returns:
        float: The cosine similarity between the two input arrays.
    """

    stop_words = stopwords.words('english')

    def preprocess(text):
        tokens = word_tokenize(text.lower())
        return [w for w in tokens if w not in stop_words]

    # Handle both strings and lists of words directly
    if isinstance(text1[0], str):
        text1_processed = [preprocess(text) for text in text1]
    else:
        text1_processed = text1
    if isinstance(text2[0], str):
        text2_processed = [preprocess(text) for text in text2]
    else:
        text2_processed = text2

    # Create common vocabulary
    all_words = set()
    for text in text1_processed + text2_processed:
        all_words.update(text)
    all_words = list(all_words)  # Convert back to a list for indexing

    # Create vectors
    def text_to_vector(text):
        vector = np.zeros(len(all_words))
        for w in text:
            if w in all_words:
                vector[all_words.index(w)] = 1
        return vector

    vector1 = text_to_vector(text1_processed)
    vector2 = text_to_vector(text2_processed)

    # Calculate cosine similarity
    cosine = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    return cosine


# Example Usage (assuming your data is already preprocessed as lists of lists)
text1 = [["I", "love", "horror", "movies"], ["The", "supernatural", "scares"]]
text2 = [["Lights", "out", "is", "horror", "movie"], ["Horror", "movies", "are", "favorite"]]

similarity = cosine_similarity(text1, text2)
print(similarity)


nan


In [3]:
def calculate_rouge(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for ref, cand in zip(references, candidates):
        score = scorer.score(ref, cand)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    # Compute average scores
    avg_scores = {key: sum(val) / len(val) for key, val in scores.items()}
    return avg_scores


In [15]:
# Load the CSV file
data = pd.read_csv('/Users/co/major_project/test_tinyllama-original.csv')

# Compute BLEU and ROUGE scores
bleu_score = calculate_bleu(data['Answer'].to_list(), data['Predicted'].to_list())
rouge_score = calculate_rouge(data['Answer'].to_list(), data['Predicted'].to_list())
similarity = cosine_similarity(data['Answer'].to_list(), data['Predicted'].to_list())

print('BLEU score:', bleu_score)
print('ROUGE score:', rouge_score)
print("Cosine Similarity", similarity)

BLEU score: 0.012533508243433479
ROUGE score: {'rouge1': 0.2831193623897731, 'rouge2': 0.07841638506130696, 'rougeL': 0.2386720187988354}
Cosine Similarity nan


# Change References

In [ ]:
import pandas as pd
import nltk
from rouge_score import rouge_scorer

# Ensure NLTK resources are available

# Function to calculate BLEU score using NLTK
def calculate_bleu(references, candidates):
    bleu_scores = []
    for ref_group, cand in zip(references, candidates):
        # Tokenize each reference in the group
        tokenized_refs = [nltk.word_tokenize(ref) for ref in ref_group]
        bleu_score = nltk.translate.bleu_score.sentence_bleu(tokenized_refs, nltk.word_tokenize(cand))
        bleu_scores.append(bleu_score)
    return sum(bleu_scores) / len(bleu_scores)

In [ ]:
# Function to calculate ROUGE score
def calculate_rouge(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for ref, cand in zip(references, candidates):
        # Use the first reference for ROUGE calculation (or could modify to use all)
        score = scorer.score(ref[0], cand)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    # Compute average scores
    avg_scores = {key: sum(val) / len(val) for key, val in scores.items()}
    return avg_scores

In [ ]:
# # Load the CSV file
# data = pd.read_csv('test_predicted (1).csv')


# # Check if 'answer1' column exists
# if 'answer1' in data.columns:
#     # Keep only rows where 'answer1' is a non-empty string
#     df = data[data['answer1'].apply(lambda x: isinstance(x, str) and x.strip() != '')]
#     # print("Rows with empty, NaN, or non-string values in 'answer1' have been dropped.")
# else:
#     # print("Column 'answer1' does not exist in the DataFrame.")
#     pass

# Load the CSV file
data = pd.read_csv('test_predicted (1).csv')

# Filter out rows where 'answer1' is not a string or is empty/Nan
data = data[data['answer1'].apply(lambda x: isinstance(x, str) and x.strip() != '')]

# Assuming 'answer1' contains multiple references separated by '||'
data['answer1'] = data['answer1'].apply(lambda x: x.split('||') if isinstance(x, str) else [])

# Prepare data for evaluation
predictions = data['generated_answer'].tolist()
references = data['answer1'].tolist()  # Now 'answer1' should be a list of references

# Now you can compute BLEU and ROUGE scores as before



# Prepare data for evaluation
predictions = data['generated_answer'].tolist()
references = [row['answer1'] for _, row in data.iterrows()]  # Assuming '||' is used to separate references

print("predictions",predictions)
print("references",references)

In [ ]:
data.columns

In [ ]:
# Compute BLEU and ROUGE scores
bleu_score = calculate_bleu(references, predictions)
rouge_score = calculate_rouge(references, predictions)

print('BLEU score:', bleu_score)
print('ROUGE score:', rouge_score)

In [ ]:
predictions[5]

In [ ]:
references[5]